In [ ]:
!pip install elevenlabs
!pip install pypdf
!pip install sounddevice soundfile
!pip install PortAudio !pip uninstall elevenlabs -y
!pip install elevenlabs --upgrade
!pip install langchain-google-genai langchain pydantic chromadb typing-extensions langchain-community 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.7/294.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 4.8 MB/s eta 0:00:00

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -y
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.4/151.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4

**BASIC SETUP**

In [ ]:
from elevenlabs import ElevenLabs

# Initialize API Client
api_key = "PUT YOUR ELEVEN LAB API KEY HERE"
client = ElevenLabs(api_key=api_key)

# Generate speech
audio_stream = client.text_to_speech.convert_as_stream(
    text="Hellow My name is Dhruvil",
    voice_id="JBFqnCBsd6RMkjVDRZzb",
    model_id="eleven_multilingual_v2"
)

# Save the audio to a file
file_path = "output.mp3"
with open(file_path, "wb") as f:
    for chunk in audio_stream:
        f.write(chunk)

print(f"Audio saved as {file_path}")

# Play the saved file (for Jupyter users)
try:
    from IPython.display import Audio
    display(Audio(file_path))
except ImportError:
    print("IPython not available, play the file manually.")


Audio saved as output.mp3


**RAG**

In [1]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import List,Dict
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_core.messages import  HumanMessage,SystemMessage
from pydantic import BaseModel
from typing_extensions import Literal
from langgraph.graph import StateGraph , START , END

In [ ]:
os.environ["GOOGLE_API_KEY"]="Enter your Gemini API here"

In [ ]:
data="/content/Stocks_Trading_For_Beginner_Traders.pdf" # YOU CAN PUT HERE YOUR DOCUMENT PATH ALSO WHATEVER YOU WANT TO APPLY RAG
loader=PyPDFLoader(data)
document=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
chunk=text_splitter.split_documents(document)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")  # Use Gemini embeddings
vectorstore = Chroma.from_documents(chunk, embedding=embeddings, persist_directory="./chroma_db")

In [4]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
prompt_template=PromptTemplate(
    template="""
    Use the following retrieved context to answer the question:
    Context: {context}
    Question: {question}
    Answer:
    """,
    input_variables=["context", "question"]
)

llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash")

qa_chain  = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
from elevenlabs import ElevenLabs
# # Function to convert text to speech using ElevenLabs API
def text_to_speech(text):
      api_key = "PUT YOUR ELEVEN LAB API KEY HERE"
      client = ElevenLabs(api_key=api_key)
      audio_stream = client.text_to_speech.convert_as_stream(
          text=text,
          voice_id="JBFqnCBsd6RMkjVDRZzb",
          model_id="eleven_multilingual_v2"
      )

      # Save the audio to a file
      file_path = "output.mp3"
      with open(file_path, "wb") as f:
          for chunk in audio_stream:
              f.write(chunk)

      print(f"Audio saved as {file_path}")

      # Play the saved file (for Jupyter users)
      try:
          from IPython.display import Audio
          display(Audio(file_path))
      except ImportError:
          print("IPython not available, play the file manually.")

# Function to handle user queries
def ask_question(query):
    print(f"👤 User: {query}")

    # Retrieve and generate response from LLM
    answer = qa_chain.invoke({"query": query})  # Ensure proper invocation
    response_text = answer["result"]  # Extract the actual response

    print(f"🤖 AI: {response_text}")

    # Convert text to speech (TTS)
    audio_file = text_to_speech(response_text)

    # Play the generated audio
    if audio_file:
        import sounddevice as sd
        audio_data, sample_rate = sf.read(audio_file)
        sd.play(audio_data, samplerate=sample_rate)
        sd.wait()  # Wait until audio is finished playing


In [7]:
user_query = input("Ask a question: ")
ask_question(user_query)

Ask a question: How THE Stock Market Works?
👤 User: How THE Stock Market Works?
🤖 AI: Companies that want to raise money sell shares on the stock market. The public can then buy and sell these shares. The company then receives the money it needs to grow.
Audio saved as output.mp3
